# Create list of event links

In [ ]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import unicodedata
import time
import re
import random
from google.colab import drive

drive.mount('/content/drive')

url = 'https://www.tapology.com/search?term=UFC&commit=Enviar&model%5Bevents%5D=eventsSearch'


#This function requests the html from the site and returns it in string form 

def get_page_info(with_user_agent: bool, url):
    user_agents = ['Mozilla/5.0 (X11; Linux x86_64; rv:86.0) Gecko/20100101 Firefox/86.0','Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246', 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36','Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9','Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1','Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36']
    if with_user_agent:
        random_user = random.randint(0,5)
        headers = {'User-Agent': user_agents[random_user]}
    else:
        headers = {}

    r = get(url, headers=headers)
    r.raise_for_status()
    if r.status_code == 503:
        print("A 503 error has occurred")
        sys.exit()
    soup = BeautifulSoup(r.text, features='html.parser')
    #print(soup)
    return str(soup)

#This function applies the gpi function to a search tapology page and returns a list of all the event links in it

def scrape_event_links(event_link):
  page = get_page_info(True, event_link)
  event_links = re.findall('/fightcenter/events/.+"',page)
  event_links = ['https://www.tapology.com'+x for x in event_links]
  event_links = [x.replace("\"" , '') for x in event_links]
  event_links = list(dict.fromkeys(event_links))
  #ESSA É UMA GAMBIARRA
  #event_links = event_links[13:]
  print(event_links)
  return event_links 

#The list of links are stored in ufc_event_links

ufc_events_links = scrape_event_links(url)


Mounted at /content/drive
['https://www.tapology.com/fightcenter/events/99416-ufc-294', 'https://www.tapology.com/fightcenter/events/99813-ufc-fight-night', 'https://www.tapology.com/fightcenter/events/98868-ufc-290', 'https://www.tapology.com/fightcenter/events/99780-ufc-fight-night', 'https://www.tapology.com/fightcenter/events/99705-ufc-fight-night', 'https://www.tapology.com/fightcenter/events/99346-ufc-289', 'https://www.tapology.com/fightcenter/events/99119-ufc-fight-night', 'https://www.tapology.com/fightcenter/events/98571-ufc-fight-night', 'https://www.tapology.com/fightcenter/events/98223-ufc-fight-night', 'https://www.tapology.com/fightcenter/events/97782-ufc-fight-night', 'https://www.tapology.com/fightcenter/events/97740-ufc-fight-night', 'https://www.tapology.com/fightcenter/events/97433-ufc-fight-night', 'https://www.tapology.com/fightcenter/events/97461-ufc-fight-night', 'https://www.tapology.com/fightcenter/events/96871-ufc-287', 'https://www.tapology.com/fightcenter/e

In [ ]:
#This function applies the gpi function to a single event link and returns a list of the bout links found in it


def scrape_bout_links(event_link):
  page = get_page_info(True, event_link)
  bout_links = re.findall('/fightcenter/bouts/.+"',page)
  bout_links = ['https://www.tapology.com'+x for x in bout_links]
  bout_links = [x.replace("\"" , '') for x in bout_links]
  bout_links = list(dict.fromkeys(bout_links))
  return bout_links

scrape_bout_links('https://www.tapology.com/fightcenter/events/92055-ufc-283')

['https://www.tapology.com/fightcenter/bouts/716441-ufc-283-glover-teixeira-vs-jamahal-sweet-dreams-hill',
 'https://www.tapology.com/fightcenter/bouts/691523-ufc-283-deiveson-deus-da-guerra-figueiredo-vs-brandon-the-assassin-baby-moreno-iv',
 'https://www.tapology.com/fightcenter/bouts/710633-ufc-283-gilbert-durinho-burns-vs-neil-haitian-sensation-magny',
 'https://www.tapology.com/fightcenter/bouts/699830-ufc-283-jessica-bate-estaca-andrade-vs-lucky-lauren-murphy',
 'https://www.tapology.com/fightcenter/bouts/696569-ufc-283-paul-bearjew-craig-vs-johnny-walker',
 'https://www.tapology.com/fightcenter/bouts/695135-ufc-283-mauricio-shogun-rua-vs-ihor-duelist-potieria',
 'https://www.tapology.com/fightcenter/bouts/724990-ufc-283-gregory-robocop-rodrigues-vs-brunno-the-hulk-ferreira',
 'https://www.tapology.com/fightcenter/bouts/724082-ufc-283-thiago-moises-vs-melquizael-melk-costa',
 'https://www.tapology.com/fightcenter/bouts/691284-ufc-283-gabriel-marretinha-bonfim-vs-mounir-the-sniper

# Scraping data from bout page

In [ ]:
#This function requests the html from a bout link and returns four lists that contain the information about the fight and the fighters.

def get_info(with_user_agent: bool, url):
    user_agents = ['Mozilla/5.0 (X11; Linux x86_64; rv:86.0) Gecko/20100101 Firefox/86.0','Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246', 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36','Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9','Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1','Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36']
    if with_user_agent:
        random_user = random.randint(0,5)
        headers = {'User-Agent': user_agents[random_user]}
    else:
        headers = {}
    r = get(url, headers=headers)
    r.raise_for_status()

    soup = BeautifulSoup(r.text, features='html.parser')
    rbout = soup.select('.navLinks span')
    rnames = soup.select('.fighterNames span')
    rinfos = soup.select('.boutComparisonTable')
    rresult = soup.select('.boutResultHolder h4')
    bout = []
    names = []
    infos = []
    result = []
    count=0
    for n in rbout:
      bout.append(n.text.strip())
    for m in rnames:
      names.append(m.text.strip())
    for l in rinfos:
      infos.append(l.text.strip())
    for n in rresult:
      n = str(n)
      n = n.replace('<h4>','')
      n = n.replace('</h4>','')
      n = n.strip()
      result.append(n)
    return bout, names, infos, result

get_info(True,'https://www.tapology.com/fightcenter/bouts/716441-ufc-283-glover-teixeira-vs-jamahal-sweet-dreams-hill')

(['UFC 283: Teixeira vs. Hill\n\xa0 | \xa0 2023.01.21\n\xa0 | \xa0\nRio de Janeiro, Rio de Janeiro, Brazil',
  'Discussion'],
 ['Jamahal Hill', 'Glover Teixeira', '"Sweet Dreams"', ''],
 ['11-1-0\n\n\xa0\nPro Record At Fight\n\xa0\n\n33-8-0\n\n\n\n\nClimbed to 12-1\n\n\xa0\nRecord After Fight\n\xa0\n\nFell to 33-9\n\n\n\n-140 (Slight Favorite)\n\xa0\nBetting Odds\n\xa0\n+120 (Near Even)\n\n\nVacant\n\xa0\nTitle\n\xa0\nVacant\n\n\n\n\nUnited States\n\n\xa0\nNationality\n\xa0\n\n\nBrazil\n\n\n\nMichigan\n\xa0\nFighting out of\n\xa0\nBethel, Connecticut\n\n\n31 years, 8 months, 2 days\n\xa0\nAge at Fight\n\xa0\n43 years, 2 months, 3 weeks, 3 days\n\n\n204.5 lbs (92.8 kgs)\n\xa0\nWeigh-In Result\n\xa0\n205.0 lbs (93.0 kgs)\n\n\n6\'4" (193cm)\n\xa0\nHeight\n\xa0\n6\'2" (188cm)\n\n\n79.0" (201cm)\n\xa0\nReach\n\xa0\n76.0" (193cm)\n\n\n\n\xa0\nGym\n\xa0\nTeixeira MMA & Fitness'],
 ['Jamahal Hill defeats Glover Teixeira via 5 Round Decision'])

# Organizing bout info and adding to lists

In [ ]:
#These two functions parse the lists returned by the gi function 
#(specifically the first and third ones, which contain the fights location and date, and the fighters tale of the tape)

def parse_bout(bout_raw_string):
  
  bout_string = bout_raw_string.replace(u'\xa0', u' ')
  bout_string = bout_string.replace(u'\n', u' ') 
  bout_string = bout_string.replace('|', ' ')
  bout_list = []
  bout_list.append(re.match('(^.+)(?:\s{2,}[0-9]+\.[0-9]+\.[0-9]+)',bout_string).groups()[0].strip())
  bout_list.append(re.search('[0-9]+\.[0-9]+\.[0-9]+',bout_string).group(0))
  bout_list.append(re.match('(?:^.+)(?:\s{3,}[0-9]+\.[0-9]+\.[0-9]+)(.*$)',bout_string).groups(0)[0].strip())
  return bout_list

def parse_infos(raw_infos):
  info_list0 = []
  for info in raw_infos:
    infos = info.replace(u'\xa0', u' ')
    infos = infos.replace(u'\n', u' ') 
    info_list0.append(infos)
  info_list = [info_list0[0].split("   ")]
  info_list = [x.strip() for x in info_list[0]]
  return info_list


#This function uses the two functions above recursively to take the html string of the bout page 
#and return four parsed lists with the information needed

def collect_lists(website):
  raw_bout_info = website[0][0]
  raw_bout_comparison = website[2]
  bout_info = parse_bout(raw_bout_info)
  bout_names = website[1]
  bout_comparison = parse_infos(raw_bout_comparison)
  bout_result = website [3]
  if 'Betting Odds' not in bout_comparison:
    bout_comparison[bout_comparison.index('Pro Record At Fight')+2:bout_comparison.index('Pro Record At Fight')+4] = [None, 'Betting Odds', None]
  return bout_info,bout_names,bout_comparison,bout_result



#This function takes the four lists as arguments and adds them to a dictionary
def infos_to_dic(bout,names,comparisons,results):
  try:
    dic = {         'F1_nickname':names[2],
                    'F2_nickname':names[3],          
                    'F1_nationality':comparisons[comparisons.index('Nationality')-1],
                    'F2_nationality':comparisons[comparisons.index('Nationality')+1],                
                    'F1_gym':comparisons[comparisons.index('Gym')-1],
                    'F2_gym':comparisons[comparisons.index('Gym')+1]}
                    
  except:
     dic = {        'F1_nickname':None,
                    'F2_nickname':None,          
                    'F1_nationality':None,
                    'F2_nationality':None,                
                    'F1_gym':None,
                    'F2_gym':None}
  try:
    dic = {'event': bout[0],
                    'date': bout[1],
                    'location':bout[2],
                    'F1_name':names[0],
                    'F2_name':names[1],
                    
                    'F1_record':comparisons[comparisons.index('Pro Record At Fight')-1],
                    'F2_record':comparisons[comparisons.index('Pro Record At Fight')+1],
                    'F1_odds':comparisons[comparisons.index('Betting Odds')-1],
                    'F2_odds':comparisons[comparisons.index('Betting Odds')+1],
                    
                    'F1_home':comparisons[comparisons.index('Fighting out of')-1],
                    'F2_home':comparisons[comparisons.index('Fighting out of')+1],
                    'F1_age':comparisons[comparisons.index('Age at Fight')-1],
                    'F2_age':comparisons[comparisons.index('Age at Fight')+1],
                    'F1_weigh_in':comparisons[comparisons.index('Weigh-In Result')-1],
                    'F2_weigh_in':comparisons[comparisons.index('Weigh-In Result')+1],
                    'F1_height':comparisons[comparisons.index('Height')-1],
                    'F2_height':comparisons[comparisons.index('Height')+1],
                    'F1_reach':comparisons[comparisons.index('Reach')-1],
                    'F2_reach':comparisons[comparisons.index('Reach')+1],
                    
                    'result':results[0]}
    print('\nDictionary inserted successfully! Result of the fight:', dic['result'])                
    return dic
  except:
    print('Error in trying to fit lists to dictionary #####', results,names,comparisons,bout)
  

#This function takes the address of the site and calls the previous funcions to return a dictionary with the bout information

def site_to_dic(address):
  site = get_info(True, address)
  site1 = collect_lists(site)
  site2 = infos_to_dic(*site1)
  return site2
site_to_dic('https://www.tapology.com/fightcenter/bouts/716441-ufc-283-glover-teixeira-vs-jamahal-sweet-dreams-hill')


Dictionary inserted successfully! Result of the fight: Jamahal Hill defeats Glover Teixeira via 5 Round Decision


{'event': 'UFC 283: Teixeira vs. Hill',
 'date': '2023.01.21',
 'location': 'Rio de Janeiro, Rio de Janeiro, Brazil',
 'F1_name': 'Jamahal Hill',
 'F2_name': 'Glover Teixeira',
 'F1_record': '11-1-0',
 'F2_record': '33-8-0',
 'F1_odds': '-140 (Slight Favorite)',
 'F2_odds': '+120 (Near Even)',
 'F1_home': 'Michigan',
 'F2_home': 'Bethel, Connecticut',
 'F1_age': '31 years, 8 months, 2 days',
 'F2_age': '43 years, 2 months, 3 weeks, 3 days',
 'F1_weigh_in': '204.5 lbs (92.8 kgs)',
 'F2_weigh_in': '205.0 lbs (93.0 kgs)',
 'F1_height': '6\'4" (193cm)',
 'F2_height': '6\'2" (188cm)',
 'F1_reach': '79.0" (201cm)',
 'F2_reach': '76.0" (193cm)',
 'result': 'Jamahal Hill defeats Glover Teixeira via 5 Round Decision'}

# Scraping data from a whole event

In [ ]:
def get_event_links(event_link):
  try:
    bout_links = scrape_bout_links(event_link)
    return bout_links
    
  except requests.exceptions.RequestException as e:
    print("An error has occurred:", e)
    
get_event_links(ufc_events_links[0])

[]

# Getting data from all UFC events at the search page

In [ ]:
# df = pd.DataFrame(columns=['event','date','location','F1_name',
#                     'F2_name','F1_nickname','F2_nickname','F1_record','F2_record','F1_odds','F2_odds',
#                     'F1_nationality','F2_nationality','F1_home','F2_home','F1_age','F2_age',
#                     'F1_weigh_in','F2_weigh_in','F1_height','F2_height','F1_reach','F2_reach','F1_gym','F2_gym','result'])
#  # Load existing data from the CSV file if it exists

# try:
#      df = pd.read_csv('/content/drive/My Drive/mma_database.csv')
# except FileNotFoundError:
#      df = pd.DataFrame(columns=['event','date','location','F1_name',
#                     'F2_name','F1_nickname','F2_nickname','F1_record','F2_record','F1_odds','F2_odds',
#                     'F1_nationality','F2_nationality','F1_home','F2_home','F1_age','F2_age',
#                     'F1_weigh_in','F2_weigh_in','F1_height','F2_height','F1_reach','F2_reach','F1_gym','F2_gym','result'])
# # Start the crawling loop
# for event in ufc_events_links:
#   df.to_csv('/content/drive/My Drive/mma_database.csv', index=False)
#   print(event)
#   time.sleep(random.randint(1,5))
#   bouts = get_event_links(event)
#   for bout in bouts:
#     time.sleep(random.randint(1,5))
#     bout_info = site_to_dic(bout)
#     df = df.append(bout_info, ignore_index=True)
#     # Save the data to the CSV file
#     df.to_csv('mma_database.csv', index=False)
#     print('\n', df.iloc[-1]['result'])


In [ ]:

df = pd.DataFrame(columns=['event','date','location','F1_name',
                   'F2_name','F1_nickname','F2_nickname','F1_record','F2_record','F1_odds','F2_odds',
                   'F1_nationality','F2_nationality','F1_home','F2_home','F1_age','F2_age',
                   'F1_weigh_in','F2_weigh_in','F1_height','F2_height','F1_reach','F2_reach','F1_gym','F2_gym','result'])

 #Load existing data from the CSV file if it exists
try:
     df = pd.read_csv('/content/drive/My Drive/mma_database.csv')
except FileNotFoundError:
     df = pd.DataFrame(columns=['event','date','location','F1_name',
                    'F2_name','F1_nickname','F2_nickname','F1_record','F2_record','F1_odds','F2_odds',
                    'F1_nationality','F2_nationality','F1_home','F2_home','F1_age','F2_age',
                    'F1_weigh_in','F2_weigh_in','F1_height','F2_height','F1_reach','F2_reach','F1_gym','F2_gym','result'])

first_link='https://www.tapology.com/fightcenter/events/93977-ufc-fight-night'
last_link='https://www.tapology.com/fightcenter/events/92055-ufc-283'

ufc_updated_links = ufc_events_links[ufc_events_links.index(first_link):ufc_events_links.index(last_link)]
for event in ufc_updated_links:
   df.to_csv('/content/drive/My Drive/mma_database.csv', index=False)
   print(event)
   if 'brawl-at-the-mall' in event or 'ufcf' in event or 'pfl' in event or 'lfa' in event or 'one' in event or 'foster' in event:
     print('Brawl at the mall detected!')
     continue
   time.sleep(random.randint(3,9))
   bouts = get_event_links(event)
   for bout in bouts:
     time.sleep(random.randint(4,8))
     bout_info = site_to_dic(bout)
     df = df.append(bout_info, ignore_index=True)
     # Save the data to the CSV file
     df.to_csv('mma_database.csv', index=False)
     
df.to_csv('/content/drive/My Drive/mma_database.csv', index=False)



https://www.tapology.com/fightcenter/events/93977-ufc-fight-night

Dictionary inserted successfully! Result of the fight: Serghei Spivac defeats Derrick Lewis via Submission at 3:05 of Round 1


<ipython-input-7-9a47c27c85e7>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(bout_info, ignore_index=True)



Dictionary inserted successfully! Result of the fight: Devin Clark defeats Da Un Jung via 3 Round Decision


<ipython-input-7-9a47c27c85e7>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(bout_info, ignore_index=True)



Dictionary inserted successfully! Result of the fight: Marcin Tybura defeats Blagoy Ivanov via 3 Round Decision


...

<ipython-input-7-9a47c27c85e7>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(bout_info, ignore_index=True)



Dictionary inserted successfully! Result of the fight: Tatsuro Taira defeats Jesus Aguilar via Submission at 4:20 of Round 1


<ipython-input-7-9a47c27c85e7>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(bout_info, ignore_index=True)


Error in trying to fit lists to dictionary ##### [] ['Ji Yeon Kim', 'Mandy Böhm', '"Fire Fist"', '"Monster"'] ['9-6-2', 'Pro Record At Fight', '7-2-0', '-280 (Moderate Favorite)', 'Betting Odds', '+230 (Moderate Underdog)', 'South Korea', 'Nationality', 'Germany', 'Las Vegas, Nevada', 'Fighting out of', 'Gelsenkirchen, Germany', '33 years, 3 months, 2 weeks, 3 days', 'Age at Fight', '33 years, 6 months, 5 days', '125.5 lbs (56.9 kgs)', 'Weigh-In Result', '125.5 lbs (56.9 kgs)', '5\'7" (170cm)', 'Height', '5\'7" (170cm)', '72.0" (183cm)', 'Reach', '71.0" (180cm)', 'MOB Training Center', 'Gym', 'SBG Ireland'] ['UFC Fight Night: Lewis vs. Spivac', '2023.02.04', 'Las Vegas, Nevada']


<ipython-input-7-9a47c27c85e7>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(bout_info, ignore_index=True)
